In [1]:
import pandas as pd
import subprocess
import logging
import re
import json
from typing import Dict
import os
import sys
import multiprocessing
PRJ_BASE = os.path.dirname(os.getcwd())
sys.path.append(PRJ_BASE)

from core.data_utils import (
    contains_only_vietnamese_letters_and_spaces,
    clean_upper_case,
    clean_words_dict,
    get_ngram_from_sentences
)

/mnt/d/personal_working/bavandangvanba/


In [2]:
from itertools import chain
def get_sentences_from_segmented_text(data):
    url, file_path, segment_file_path = data
    tmp_sentences = []
    abs_segment_file_path = base_path + segment_file_path
    with open(abs_segment_file_path, 'r') as f:
        text = f.read()
    for line in text.split('\n\n'):
        sentence = []
        for tmp_text in line.replace('_',' ').replace('\t',' ').split('\n'):
            if tmp_text:
                cleaned_word = tmp_text[2:].strip()
                if " " not in cleaned_word or not all(word[0].isupper() for word in cleaned_word.split()):
                    cleaned_word = cleaned_word.lower()
                else:
                    cleaned_word = tmp_text[2:].strip()
                if not contains_only_vietnamese_letters_and_spaces(cleaned_word):
                    if sentence:
                        tmp_sentences.append(sentence)
                    sentence=[]
                    continue

                sentence.append(cleaned_word)
        if sentence:
            tmp_sentences.append(sentence)
    return tmp_sentences



# data from song

In [3]:
base_path = os.path.join(PRJ_BASE,'data/song_from_lyricvn_com/')
loaded_pd_songs = pd.read_csv(base_path + 'songs_meta_data.csv')
logging.info(f'Start extracting sentences for: {len(loaded_pd_songs)} files')
with multiprocessing.Pool(processes=10) as pool:
    results = pool.map(get_sentences_from_segmented_text, loaded_pd_songs.values)
sentences = list(chain.from_iterable(results))
logging.info(f'Done extracting sentences for: {len(loaded_pd_songs)} files')
n_gram_phrases = clean_upper_case(get_ngram_from_sentences(sentences))
pd.to_pickle(n_gram_phrases, os.path.join(PRJ_BASE,'tmp/dictionary_from_songs_data.pkl'))
len(n_gram_phrases)


2023-11-11 04:42:39 INFO     Start extracting sentences for: 16203 files
2023-11-11 04:43:19 INFO     Done extracting sentences for: 16203 files


256336

# data from tang thu vien

In [4]:
base_path = os.path.join(PRJ_BASE)
loaded_pd_truyen = pd.read_csv(base_path + '/data/truyen_from_tangthuvien_vn/truyen_meta_data.csv')[['truyen_url','file_path','segment_file_path']]
base_path += '/'

logging.info(f'Start extracting sentences for: {len(loaded_pd_truyen)} files')
with multiprocessing.Pool(processes=10) as pool:
    results = pool.map(get_sentences_from_segmented_text, loaded_pd_truyen.values)
sentences = list(chain.from_iterable(results))
logging.info(f'Done extracting sentences for: {len(loaded_pd_truyen)} files')
n_gram_phrases = clean_upper_case(get_ngram_from_sentences(sentences))

pd.to_pickle(n_gram_phrases, os.path.join(PRJ_BASE,'tmp/dictionary_from_truyen_data.pkl'))
len(n_gram_phrases)

2023-11-11 04:43:22 INFO     Start extracting sentences for: 22652 files
2023-11-11 04:47:40 INFO     Done extracting sentences for: 22652 files


1300826